# 1. Data Cleaning and Feature Selection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
from google.colab import files
import zipfile
import os

In [ ]:
# Upload and load data
uploaded1 = files.upload()
file_path = list(uploaded1.keys())[0]
df = pd.read_csv(file_path)



In [ ]:
# Display dataset info
print("\nDataset Information:")
df.info()
print("\nSummary Statistics:\n", df.describe())



In [ ]:
# Load dataset
#file_path = "/Users/ng/Downloads/Latest Dataset.xlsx"  # Ensure the correct path on your local machine

In [ ]:
# Load the dataset and check available sheets
#xls = pd.ExcelFile(file_path)
#print("Available Sheets:", xls.sheet_names)

In [ ]:
# Load the dataset from the identified sheet
df = pd.read_excel(xls, sheet_name="COVID-19_Reported_Patient_Impac")

In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()

In [ ]:
# Drop completely empty or redundant columns
drop_columns = [
    "geocoded_state",  # Completely empty column
    "inpatient_beds_utilization_numerator", "inpatient_beds_utilization_denominator", # Redundant
    "adult_icu_bed_utilization_numerator", "adult_icu_bed_utilization_denominator"
]
df = df.drop(columns=drop_columns, errors='ignore')


In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()

In [ ]:
# Drop pediatric-related columns
pediatric_columns = [
    "previous_day_admission_pediatric_covid_confirmed", "previous_day_admission_pediatric_covid_suspected",
    "previous_day_admission_pediatric_covid_confirmed_0_4", "previous_day_admission_pediatric_covid_confirmed_5_11",
    "previous_day_admission_pediatric_covid_confirmed_12_17", "previous_day_admission_pediatric_covid_confirmed_unknown",
    "staffed_icu_pediatric_patients_confirmed_covid", "staffed_pediatric_icu_bed_occupancy",
    "total_staffed_pediatric_icu_beds", "all_pediatric_inpatient_bed_occupied", "all_pediatric_inpatient_beds"
]
df = df.drop(columns=pediatric_columns, errors='ignore')


In [ ]:
# Drop columns with more than 40% missing values
missing_threshold = 0.4 * len(df)
df = df.dropna(thresh=missing_threshold, axis=1)

In [ ]:
# Drop the original detailed previous day admission columns
drop_admission_columns = [
    "previous_day_admission_adult_covid_confirmed_18-19", "previous_day_admission_adult_covid_confirmed_20-29",
    "previous_day_admission_adult_covid_confirmed_30-39", "previous_day_admission_adult_covid_confirmed_40-49",
    "previous_day_admission_adult_covid_confirmed_50-59", "previous_day_admission_adult_covid_confirmed_60-69",
    "previous_day_admission_adult_covid_confirmed_70-79", "previous_day_admission_adult_covid_confirmed_80+"
]
df = df.drop(columns=drop_admission_columns, errors='ignore')


In [ ]:
# Fill missing values
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

In [ ]:
# Convert date column into useful features
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['week_of_year'] = df['date'].dt.isocalendar().week
df = df.drop(columns=['date'])  # Drop original date column after feature extraction


In [ ]:
# Drop highly correlated redundant features based on correlation matrix
high_corr_features = [
    "inpatient_beds_used",  # Correlated with inpatient_beds_utilization
    "staffed_adult_icu_bed_occupancy",  # Correlated with adult_icu_bed_utilization
    "staffed_icu_adult_patients_confirmed_covid",  # Correlated with total_adult_patients_hospitalized_confirmed_covid
    "percent_of_inpatients_with_covid_numerator", "percent_of_inpatients_with_covid_denominator", # Redundant
    "inpatient_bed_covid_utilization_numerator", "inpatient_bed_covid_utilization_denominator"
    "total_previous_day_admissions",  # Replaced with age-grouped admissions
    "adult_icu_bed_covid_utilization_numerator"  # Redundant with ICU utilization
    "total_adult_patients_hospitalized_confirmed_and_suspected_covid"
    "staffed_icu_adult_patients_confirmed_and_suspected_covid", # Redundant with total_adult_patients_hospitalized_confirmed_covid
    "total_staffed_adult_icu_beds"  # Redundant as utilization metrics are retained
]
df = df.drop(columns=high_corr_features, errors='ignore')

In [ ]:
# Drop redundant staffing shortage columns (keep only 'critical_staffing_shortage_today_yes')
staffing_columns = [
    "critical_staffing_shortage_today_no", "critical_staffing_shortage_today_not_reported",
    "critical_staffing_shortage_anticipated_within_week_yes", "critical_staffing_shortage_anticipated_within_week_no",
    "critical_staffing_shortage_anticipated_within_week_not_reported"
]
df = df.drop(columns=staffing_columns, errors='ignore')

In [ ]:
# Drop therapeutic supply-related columns (not useful for bed utilization prediction)
therapeutic_columns = [
    "on_hand_supply_therapeutic_a_casirivimab_imdevimab_courses",
    "on_hand_supply_therapeutic_b_bamlanivimab_courses",
    "on_hand_supply_therapeutic_c_bamlanivimab_etesevimab_courses",
    "previous_week_therapeutic_a_casirivimab_imdevimab_courses_used",
    "previous_week_therapeutic_b_bamlanivimab_courses_used",
    "previous_week_therapeutic_c_bamlanivimab_etesevimab_courses_used"
]
df = df.drop(columns=therapeutic_columns, errors='ignore')

In [ ]:

# Perform Variance Inflation Factor (VIF) check for multicollinearity
X_vif = df.select_dtypes(include=['float64', 'int64'])
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF Score"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
print("\nVariance Inflation Factor (VIF) Scores:")
print(vif_data.sort_values(by="VIF Score", ascending=False))

In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()

In [ ]:
# Feature Importance using Mutual Information
target_variable = "inpatient_beds_utilization"
numeric_df_filled = df.select_dtypes(include=['float64', 'int64']).drop(columns=[target_variable], errors="ignore")
imputer = SimpleImputer(strategy="median")
X = imputer.fit_transform(numeric_df_filled)
y = df[target_variable].fillna(df[target_variable].median())

mi_scores = mutual_info_regression(X, y)
mi_scores_df = pd.Series(mi_scores, index=numeric_df_filled.columns).sort_values(ascending=False)
print("\nTop 20 Important Features (Mutual Information):")
print(mi_scores_df.head(20))


In [ ]:
# Recalculate numeric columns after dropping features
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Normalize numerical features
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


# Save the cleaned dataset
df.to_csv("cleaned_hospital_data.csv", index=False)
print("\n✅ Data Cleaning, Feature Selection, Feature Importance & Normalization Completed!")


# 2. EDA and Trend Analysis

In [ ]:
import pandas as pd
import missingno as msno  # Install with `pip install missingno`
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# Aggregate ICU occupancy by state
state_summary = df.groupby("state")[["staffed_adult_icu_bed_occupancy"]].mean().reset_index()

# State abbreviation mapping
us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}
# Reverse mapping: Full name → Abbreviation
state_abbrev_reverse = {v: k for k, v in us_state_abbrev.items()}

# Convert state names to full names for merging
state_summary["state"] = state_summary["state"].map(us_state_abbrev).fillna(state_summary["state"])

# Load US states shapefile
usa = gpd.read_file("https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json")

# Rename column for merging
usa = usa.rename(columns={"name": "state"})

# Merge ICU occupancy data with the map
usa = usa.merge(state_summary, on="state", how="left")

# Add state abbreviations for labeling
usa["state_code"] = usa["state"].map(state_abbrev_reverse)

# Compute state centroids for label placement
usa["centroid"] = usa.geometry.centroid

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(14, 8))
usa.boundary.plot(ax=ax, linewidth=1, color="black")  # State borders
usa.plot(column="staffed_adult_icu_bed_occupancy", cmap="Reds", linewidth=0.8, edgecolor='black', legend=True, ax=ax)

# Add state abbreviation labels at centroid positions
for idx, row in usa.iterrows():
    if row["centroid"] and row["state_code"]:
        plt.text(row["centroid"].x, row["centroid"].y, row["state_code"], fontsize=8, ha="center", color="black")

# Add title
plt.title("ICU Bed Occupancy by State", fontsize=14)

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Ensure 'date' is in datetime format
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])

# Aggregate data by date (ignoring state for cleaner visualization)
df_grouped = df_cleaned.groupby('date').sum()

# Time Series - Hospital and ICU Bed Utilization
plt.figure(figsize=(12, 6))
plt.plot(df_grouped.index, df_grouped['inpatient_beds_utilization'], label='Inpatient Bed Utilization', linestyle='-', marker='o')
plt.plot(df_grouped.index, df_grouped['adult_icu_bed_utilization'], label='Adult ICU Bed Utilization', linestyle='--', marker='s')
plt.plot(df_grouped.index, df_grouped['adult_icu_bed_covid_utilization'], label='COVID ICU Utilization', linestyle='-.', marker='^')
plt.xlabel("Date")
plt.ylabel("Utilization Rate")
plt.title("Hospital Bed Utilization Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Stacked Area Chart - COVID vs. Non-COVID Utilization
plt.figure(figsize=(12, 6))
plt.stackplot(df_grouped.index,
              df_grouped['inpatient_beds_used'] - df_grouped['inpatient_beds_used_covid'],
              df_grouped['inpatient_beds_used_covid'],
              labels=['Non-COVID Inpatient Beds Used', 'COVID Inpatient Beds Used'],
              alpha=0.7)
plt.xlabel("Date")
plt.ylabel("Number of Beds Used")
plt.title("Hospital Bed Usage: COVID vs. Non-COVID")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Histogram (Distribution of ICU Bed Occupancy)
# Identifies skewness and potential need for transformation
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['staffed_adult_icu_bed_occupancy'], bins=30, kde=True, color='blue')
plt.xlabel("Number of Occupied Adult ICU Beds")
plt.ylabel("Frequency")
plt.title("Distribution of Staffed Adult ICU Bed Occupancy")
plt.show()

In [ ]:
# Define the new target variable
y_target = 'staffed_adult_icu_bed_occupancy'

# Selecting relevant columns for correlation analysis, ensuring target variable is included
corr_columns = [
    'inpatient_beds', 'inpatient_beds_used', 'inpatient_beds_used_covid',
    'total_staffed_adult_icu_beds', 'staffed_adult_icu_bed_occupancy',
    'total_staffed_pediatric_icu_beds', 'staffed_pediatric_icu_bed_occupancy',
    'all_pediatric_inpatient_bed_occupied', 'all_pediatric_inpatient_beds',
    'inpatient_beds_utilization', 'adult_icu_bed_utilization',
    'adult_icu_bed_covid_utilization', 'inpatient_bed_covid_utilization'
]

# Compute correlation matrix
correlation_matrix = df_cleaned[corr_columns].corr()

# Plot updated correlation heatmap with new Y target variable
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Hospital Resources (Target: Staffed Adult ICU Bed Occupancy)")
plt.show()

# Identify top correlated features with the new target variable
top_correlated_features = correlation_matrix[y_target].abs().sort_values(ascending=False)

# Print top correlated features
print("Top correlated features with staffed adult ICU bed occupancy:")
print(top_correlated_features)

In [ ]:
#  COVID-19 Hospitalized Patients Over Time
plt.figure(figsize=(12, 6))
plt.plot(df_grouped.index, df_grouped['total_adult_patients_hospitalized_confirmed_and_suspected_covid'],
         label='Total Adult COVID Patients', linestyle='-', marker='o')
plt.plot(df_grouped.index, df_grouped['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid'],
         label='Total Pediatric COVID Patients', linestyle='--', marker='s')
plt.xlabel("Date")
plt.ylabel("Number of COVID Patients")
plt.title("COVID-19 Hospitalized Patients Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# COVID-19 Patient Data Correlation Heatmap
covid_corr_columns = [
    'total_adult_patients_hospitalized_confirmed_and_suspected_covid',
    'total_adult_patients_hospitalized_confirmed_covid',
    'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid',
    'total_pediatric_patients_hospitalized_confirmed_covid',
    'staffed_icu_adult_patients_confirmed_and_suspected_covid',
    'staffed_icu_adult_patients_confirmed_covid',
    'staffed_icu_pediatric_patients_confirmed_covid',
    'percent_of_inpatients_with_covid', 'hospital_onset_covid'
]

correlation_matrix = df_cleaned[covid_corr_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of COVID-19 Patient Statistics")
plt.show()

In [ ]:
#  COVID-19 Admissions Over Time
plt.figure(figsize=(12, 6))
plt.plot(df_grouped.index, df_grouped['previous_day_admission_adult_covid_confirmed'],
         label='Adult Confirmed Admissions', linestyle='-', marker='o')
plt.plot(df_grouped.index, df_grouped['previous_day_admission_adult_covid_suspected'],
         label='Adult Suspected Admissions', linestyle='--', marker='s')
plt.plot(df_grouped.index, df_grouped['previous_day_admission_pediatric_covid_confirmed_0_4'] +
         df_grouped['previous_day_admission_pediatric_covid_confirmed_5_11'] +
         df_grouped['previous_day_admission_pediatric_covid_confirmed_12_17'],
         label='Total Pediatric Admissions', linestyle='-.', marker='^')
plt.xlabel("Date")
plt.ylabel("Number of COVID Admissions")
plt.title("COVID-19 Admissions Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Age Distribution of COVID-19 Admissions
# Define the mapping of column names to clean age labels
age_groups = {
    'previous_day_admission_adult_covid_confirmed_18-19': '18-19',
    'previous_day_admission_adult_covid_confirmed_20-29': '20-29',
    'previous_day_admission_adult_covid_confirmed_30-39': '30-39',
    'previous_day_admission_adult_covid_confirmed_40-49': '40-49',
    'previous_day_admission_adult_covid_confirmed_50-59': '50-59',
    'previous_day_admission_adult_covid_confirmed_60-69': '60-69',
    'previous_day_admission_adult_covid_confirmed_70-79': '70-79',
    'previous_day_admission_adult_covid_confirmed_80+': '80+'
}

# Extract relevant data and rename columns
df_age_grouped = df_grouped[list(age_groups.keys())].mean().rename(index=age_groups)

plt.figure(figsize=(10, 6))

# Plot with cleaned x-axis labels
plt.bar(df_age_grouped.index, df_age_grouped.values, color='c')

# Improve labels and title
plt.xlabel("Age Group")
plt.ylabel("Average Daily Admissions")
plt.title("Average Daily COVID-19 Admissions by Age Group")

# Adjust x-axis for readability
plt.xticks(rotation=0)  # Keeps labels horizontal for clarity

# Add grid for better visualization
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

In [ ]:
# Regional Differences in COVID-19 Admissions
df_statewise = df_cleaned.groupby('state')[['previous_day_admission_adult_covid_confirmed']].sum()

df_statewise.sort_values('previous_day_admission_adult_covid_confirmed', ascending=False).head(10).plot(kind='bar', figsize=(12, 6))
plt.xlabel("State")
plt.ylabel("Total COVID Admissions")
plt.title("Top 10 States by COVID-19 Admissions")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# COVID-19 Admissions Correlation Heatmap
covid_admissions_corr_columns = [
    'previous_day_admission_adult_covid_confirmed',
    'previous_day_admission_adult_covid_suspected',
    'previous_day_admission_adult_covid_confirmed_18-19',
    'previous_day_admission_adult_covid_confirmed_20-29',
    'previous_day_admission_adult_covid_confirmed_30-39',
    'previous_day_admission_adult_covid_confirmed_40-49',
    'previous_day_admission_adult_covid_confirmed_50-59',
    'previous_day_admission_adult_covid_confirmed_60-69',
    'previous_day_admission_adult_covid_confirmed_70-79',
    'previous_day_admission_adult_covid_confirmed_80+'
]

correlation_matrix = df_cleaned[covid_admissions_corr_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of COVID-19 Admissions by Age Group")
plt.show()

In [ ]:
# Staffing Shortages Over Time
plt.figure(figsize=(12, 6))
plt.plot(df_grouped.index, df_grouped['critical_staffing_shortage_today_yes'],
         label='Current Staffing Shortage', linestyle='-', marker='o')
plt.plot(df_grouped.index, df_grouped['critical_staffing_shortage_anticipated_within_week_yes'],
         label='Anticipated Staffing Shortage', linestyle='--', marker='s')
plt.xlabel("Date")
plt.ylabel("Number of Hospitals Reporting Shortages")
plt.title("Staffing Shortages Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Regional Differences in Staffing Shortages
df_statewise = df_cleaned.groupby('state')[['critical_staffing_shortage_today_yes',
                                            'critical_staffing_shortage_anticipated_within_week_yes']].sum()

df_statewise.sort_values('critical_staffing_shortage_today_yes', ascending=False).head(10).plot(kind='bar', figsize=(12, 6))
plt.xlabel("State")
plt.ylabel("Total Staffing Shortages Reported")
plt.title("Top 10 States by Staffing Shortages")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Correlation Between Staffing Shortages & Hospital Strain
staffing_corr_columns = [
    'critical_staffing_shortage_today_yes',
    'critical_staffing_shortage_anticipated_within_week_yes',
    'staffed_adult_icu_bed_occupancy',
    'percent_of_inpatients_with_covid',
    'previous_day_admission_adult_covid_confirmed'
]

correlation_matrix = df_cleaned[staffing_corr_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap: Staffing Shortages & Hospital Strain")
plt.show()

In [ ]:
# COVID-19 & Influenza Deaths Over Time
plt.figure(figsize=(12, 6))
plt.plot(df_grouped.index, df_grouped['deaths_covid'],
         label='COVID Deaths', linestyle='-', marker='o', color='red')
plt.plot(df_grouped.index, df_grouped['previous_day_deaths_covid_and_influenza'],
         label='COVID + Influenza Deaths', linestyle='--', marker='s', color='purple')
plt.plot(df_grouped.index, df_grouped['previous_day_deaths_influenza'],
         label='Influenza Deaths', linestyle='-.', marker='^', color='blue')
plt.xlabel("Date")
plt.ylabel("Number of Deaths")
plt.title("COVID-19 & Influenza Deaths Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# COVID-19 vs. Influenza: Proportion of Total Deaths

import matplotlib.pyplot as plt

# Define new labels
death_totals = df_cleaned[['deaths_covid', 'previous_day_deaths_covid_and_influenza', 'previous_day_deaths_influenza']].sum()
labels = ['COVID-19 Deaths', 'COVID + Influenza Deaths', 'Influenza Deaths']

# Define colors
colors = ['pink', 'red', 'orange']

plt.figure(figsize=(8, 6))

# Plot pie chart with labels outside
wedges, texts, autotexts = plt.pie(
    death_totals, labels=labels, autopct='%1.1f%%', colors=colors,
    pctdistance=0.85, labeldistance=1.1, wedgeprops={'edgecolor': 'none'}
)

# Improve label readability
for text in texts:
    text.set_fontsize(10)
for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_color('black')

plt.title("Proportion of Deaths: COVID-19 vs. Influenza")
plt.ylabel("")  # Hide y-label

plt.show()

In [ ]:
# Influenza Hospitalizations Over Time
plt.figure(figsize=(12, 6))
plt.plot(df_grouped.index, df_grouped['total_patients_hospitalized_confirmed_influenza'],
         label='Influenza Hospitalized Patients', linestyle='-', marker='o', color='blue')
plt.plot(df_grouped.index, df_grouped['total_patients_hospitalized_confirmed_influenza_and_covid'],
         label='Influenza + COVID Hospitalized Patients', linestyle='--', marker='s', color='purple')
plt.xlabel("Date")
plt.ylabel("Number of Hospitalized Patients")
plt.title("Influenza Hospitalizations Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Influenza vs. COVID-19 ICU Burden
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_cleaned['staffed_adult_icu_bed_occupancy'],
                y=df_cleaned['icu_patients_confirmed_influenza'],
                alpha=0.5, color='blue', label='Influenza ICU Patients')
sns.scatterplot(x=df_cleaned['staffed_adult_icu_bed_occupancy'],
                y=df_cleaned['staffed_icu_adult_patients_confirmed_covid'],
                alpha=0.5, color='red', label='COVID ICU Patients')
plt.xlabel("Total ICU Bed Occupancy")
plt.ylabel("Number of ICU Patients")
plt.title("Influenza vs. COVID-19 ICU Burden")
plt.legend()
plt.show()

In [ ]:
# Influenza Admissions Correlation Heatmap
influenza_corr_columns = [
    'icu_patients_confirmed_influenza',
    'previous_day_admission_influenza_confirmed',
    'total_patients_hospitalized_confirmed_influenza',
    'total_patients_hospitalized_confirmed_influenza_and_covid',
    'staffed_adult_icu_bed_occupancy',
    'staffed_icu_adult_patients_confirmed_covid'
]

correlation_matrix = df_cleaned[influenza_corr_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap: Influenza, COVID, and ICU Usage")
plt.show()

In [ ]:
# Pediatric COVID-19 Hospitalizations Over Time
plt.figure(figsize=(12, 6))
plt.plot(df_grouped.index, df_grouped['total_pediatric_patients_hospitalized_confirmed_covid'],
         label='Total Pediatric COVID Hospitalizations', linestyle='-', marker='o', color='blue')
plt.xlabel("Date")
plt.ylabel("Number of Hospitalized Pediatric Patients")
plt.title("Pediatric COVID-19 Hospitalizations Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Pediatric Admissions by Age Group
age_groups = {
    'previous_day_admission_pediatric_covid_confirmed_0_4': '0-4',
    'previous_day_admission_pediatric_covid_confirmed_5_11': '5-11',
    'previous_day_admission_pediatric_covid_confirmed_12_17': '12-17'
}

# Select and rename the columns
df_pediatric = df_grouped[list(age_groups.keys())].rename(columns=age_groups)

# Plot the bar chart
df_pediatric.mean().plot(kind='bar', figsize=(8, 6), color=['blue', 'purple', 'green'])
plt.xlabel("Age Group")
plt.ylabel("Average Daily Admissions")
plt.title("Average Daily Pediatric COVID Admissions by Age Group")
plt.xticks(rotation=0)  # Keep labels horizontal for better readability
plt.show()

In [ ]:
# Pediatric ICU vs. General Pediatric Hospitalizations
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_cleaned['total_pediatric_patients_hospitalized_confirmed_covid'],
                y=df_cleaned['staffed_icu_pediatric_patients_confirmed_covid'],
                alpha=0.5, color='red')
plt.xlabel("Total Pediatric Hospitalizations")
plt.ylabel("Pediatric ICU COVID Patients")
plt.title("Pediatric ICU vs. Total COVID Hospitalizations")
plt.show()

In [ ]:
# Pediatric COVID Correlation Heatmap
pediatric_corr_columns = [
    'total_pediatric_patients_hospitalized_confirmed_covid',
    'previous_day_admission_pediatric_covid_confirmed_0_4',
    'previous_day_admission_pediatric_covid_confirmed_5_11',
    'previous_day_admission_pediatric_covid_confirmed_12_17',
    'staffed_icu_pediatric_patients_confirmed_covid',
    'total_adult_patients_hospitalized_confirmed_covid'
]

correlation_matrix = df_cleaned[pediatric_corr_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap: Pediatric COVID Admissions & ICU Usage")
plt.show()

In [ ]:

# Feature Engineering: Add lag features and moving averages

# Creating lag variables for past ICU occupancy trends
for lag in [1, 7, 14]:  # Lags of 1 day, 7 days, and 14 days
    df_filtered[f"ICU_lag_{lag}"] = df_filtered["staffed_adult_icu_bed_occupancy"].shift(lag)

# Adding moving averages to smooth trends
df_filtered["ICU_7day_avg"] = df_filtered["staffed_adult_icu_bed_occupancy"].rolling(window=7).mean()

# Drop any rows with NaN values from lag features
df_filtered.dropna(inplace=True)

# Display transformed dataset
df_filtered.head()


**State level categorization for Policy Planning**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
from google.colab import files
import zipfile
import os
import pandas as pd

In [ ]:
uploaded1= files.upload()


In [ ]:
# Load Data
file_path = list(uploaded1.keys())[0]
df = pd.read_csv(file_path)

In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()
print("\nSummary Statistics:\n", df.describe())

In [ ]:
# Drop completely empty or redundant columns
drop_columns = [
    "geocoded_state",  # Completely empty column
    "inpatient_beds_utilization_numerator", "inpatient_beds_utilization_denominator", # Redundant
    "adult_icu_bed_utilization_numerator", "adult_icu_bed_utilization_denominator"
]
df = df.drop(columns=drop_columns, errors='ignore')


In [ ]:
# Drop the original detailed previous day admission columns
drop_admission_columns = [
    "previous_day_admission_adult_covid_confirmed_18-19", "previous_day_admission_adult_covid_confirmed_20-29",
    "previous_day_admission_adult_covid_confirmed_30-39", "previous_day_admission_adult_covid_confirmed_40-49",
    "previous_day_admission_adult_covid_confirmed_50-59", "previous_day_admission_adult_covid_confirmed_60-69",
    "previous_day_admission_adult_covid_confirmed_70-79", "previous_day_admission_adult_covid_confirmed_80+"
]
df = df.drop(columns=drop_admission_columns, errors='ignore')


In [ ]:
# Fill missing values
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

In [ ]:
# Convert 'date' column and extract temporal features
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['week_of_year'] = df['date'].dt.isocalendar().week
df = df.sort_values('date')



In [ ]:
numeric_df = df.select_dtypes(include=['number'])

# Compute correlation
corr_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, cmap="coolwarm", annot=False, fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

**State-Level Categorization for Policy Planning**

In [ ]:
# Renaming or identifying correct column names
columns_needed = [
    "inpatient_beds_utilization",
    "critical_staffing_shortage_today_yes",
    "critical_staffing_shortage_today_no",
    "percent_of_inpatients_with_covid"
]


In [ ]:
# Check for missing columns
missing_cols = [col for col in columns_needed if col not in df.columns]
if missing_cols:
    print(f"⚠️ Missing columns in dataset: {missing_cols}")
else:
    # Calculate staffing shortage ratio
    df["staffing_shortage_ratio"] = df["critical_staffing_shortage_today_yes"] / (
        df["critical_staffing_shortage_today_yes"] + df["critical_staffing_shortage_today_no"]
    )

In [ ]:
  # Convert percentages if needed
df["inpatient_beds_utilization"] *= 100
df["staffing_shortage_ratio"] *= 100
df["percent_of_inpatients_with_covid"] *= 100

In [ ]:
# 1️⃣ Categorize Hospital Pressure
def categorize_hospital_pressure(value):
    if value < 60:
        return "Green (Low)"
    elif 60 <= value <= 80:
        return "Yellow (Moderate)"
    else:
        return "Red (High)"

df["Hospital_Pressure_Category"] = df["inpatient_beds_utilization"].apply(categorize_hospital_pressure)


In [ ]:
# 2️⃣ Categorize Staffing Crisis
def categorize_staffing(value):
    if value < 20:
        return "Low"
    elif 20 <= value <= 50:
        return "Moderate"
    else:
        return "High"

df["Staffing_Crisis_Level"] = df["staffing_shortage_ratio"].apply(categorize_staffing)


In [ ]:
# 3️⃣ Categorize COVID-19 Burden
def categorize_covid_burden(value):
    if value < 5:
         return "Low"
    elif 5 <= value <= 15:
         return "Moderate"
    else:
         return "High"

df["COVID_Burden_Level"] = df["percent_of_inpatients_with_covid"].apply(categorize_covid_burden)


In [ ]:
 # ✅ Print Summary
print(df[["Hospital_Pressure_Category", "Staffing_Crisis_Level", "COVID_Burden_Level"]].value_counts())

In [ ]:
# Saving the preprocessed DataFrame to a CSV file
from google.colab import files  # If using Colab

def save_preprocessed_data(df, filename='preprocessed_data.csv'):
    try:
        df.to_csv(filename, index=False)
        print(f'Preprocessed data saved as {filename}')
        files.download(filename)
    except Exception as e:
        print(f'Error saving file: {e}')

# Updated Plotting Function with Custom Mapping
import matplotlib.pyplot as plt
import seaborn as sns

def plot_percentage_count_plots(df, categorical_features):
    # Define raw-to-clean value mapping
    label_map = {
        'Red (High)': 'High',
        'Yellow (Moderate)': 'Moderate',
        'Green (Low)': 'Low'
    }

    # Color mapping
    custom_palette = {
        'High': 'red',
        'Moderate': 'gold',
        'Low': 'green'
    }

    fig, axes = plt.subplots(1, len(categorical_features), figsize=(6 * len(categorical_features), 5))

    if len(categorical_features) == 1:
        axes = [axes]

    for idx, feature in enumerate(categorical_features):
        # Clean and remap values
        df[feature] = df[feature].astype(str).str.strip().str.title()
        df[feature] = df[feature].replace(label_map)

        # Compute percentages
        value_counts = df[feature].value_counts(normalize=True) * 100
        ordered = ['Low', 'Moderate', 'High']
        value_counts = value_counts.reindex([x for x in ordered if x in value_counts.index])

        # Plot
        sns.barplot(
            x=value_counts.index,
            y=value_counts.values,
            palette=[custom_palette.get(x, 'gray') for x in value_counts.index],
            ax=axes[idx]
        )

        axes[idx].set_title(f'Percentage Plot for {feature}')
        axes[idx].set_xlabel(feature)
        axes[idx].set_ylabel('Percentage (%)')
        axes[idx].tick_params(axis='x', rotation=45)

        # Annotate
        for i, p in enumerate(value_counts.values):
            axes[idx].text(i, p + 1, f'{p:.1f}%', ha='center')

    plt.tight_layout()
    plt.show()

# Example usage
save_preprocessed_data(df)
categorical_columns = ['Hospital_Pressure_Category', 'Staffing_Crisis_Level', 'COVID_Burden_Level']
plot_percentage_count_plots(df, categorical_columns)


In [ ]:
# 🔹 1. Line Plot: Trends in Hospital Bed Utilization
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x=df.index, y="inpatient_beds_utilization", label="Inpatient Bed Utilization")
sns.lineplot(data=df, x=df.index, y="staffing_shortage_ratio", label="Staffing Shortage Ratio", linestyle="dashed")
sns.lineplot(data=df, x=df.index, y="percent_of_inpatients_with_covid", label="COVID-19 Inpatients %", linestyle="dotted")
plt.xlabel("Records")
plt.ylabel("Percentage")
plt.title("Hospital Bed Utilization and Staffing Shortages Over Time")
plt.legend()
plt.show()

In [ ]:
# 🔹 2. Scatter Plot: Staffing Shortage vs. COVID-19 Burden
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df["staffing_shortage_ratio"], y=df["percent_of_inpatients_with_covid"], hue=df["Hospital_Pressure_Category"], palette="coolwarm")
plt.xlabel("Staffing Shortage Ratio (%)")
plt.ylabel("COVID-19 Inpatients (%)")
plt.title("Staffing Shortage vs COVID-19 Burden")
plt.show()

In [ ]:
# 🔹 3. Correlation Heatmap
plt.figure(figsize=(10, 6))
corr_matrix = df[["inpatient_beds_utilization", "staffing_shortage_ratio", "percent_of_inpatients_with_covid"]].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# 🔹 4. Stacked Bar Chart: Hospital Categories
category_counts = df.groupby(["Hospital_Pressure_Category", "Staffing_Crisis_Level"])["COVID_Burden_Level"].count().unstack()
category_counts.plot(kind="bar", stacked=True, figsize=(10, 6), colormap="viridis")
plt.xlabel("Hospital Pressure & Staffing Crisis")
plt.ylabel("Count")
plt.title("Stacked Bar Chart of Hospital Categories")
plt.legend(title="COVID Burden Level")
plt.show()

In [ ]:
# 🔹 5. Stacked Bar Chart: Hospital Categories (Percentage)
import matplotlib.pyplot as plt

# Group and normalize counts to get percentages
category_counts = df.groupby(["Hospital_Pressure_Category", "Staffing_Crisis_Level"])["COVID_Burden_Level"].count().unstack()

# Convert counts to row-wise percentages
category_percentages = category_counts.div(category_counts.sum(axis=1), axis=0) * 100

# Plot stacked bar chart with percentages
category_percentages.plot(kind="bar", stacked=True, figsize=(10, 6), colormap="viridis")

plt.xlabel("Hospital Pressure & Staffing Crisis")
plt.ylabel("Percentage (%)")
plt.title("Stacked Bar Chart of Hospital Categories (by COVID Burden Level %)")
plt.legend(title="COVID Burden Level")
plt.tight_layout()
plt.show()


# Scenario Planning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import joblib

In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])
df['week_of_year'] = df['date'].apply(lambda x: pd.to_datetime(x).isocalendar().week)
df['year'] = df['date'].dt.year

In [ ]:
df['staff_shortage_pct'] = (
    df['critical_staffing_shortage_today_yes'] /
    (df['critical_staffing_shortage_today_yes'] + df['critical_staffing_shortage_today_no'])
) * 100

In [ ]:
# Encode states as integers
le = LabelEncoder()
df['state_encoded'] = le.fit_transform(df['state'])

In [ ]:
# Feature engineering
df['admission_rate'] = np.random.randint(5, 25, size=len(df))
df['emergency_surge'] = np.random.choice([0.0, 0.1, 0.2, 0.3], size=len(df))
df['staff_current_shortage'] = np.random.choice([0, 1], size=len(df))
df['staff_anticipated_shortage'] = np.random.choice([0, 1], size=len(df))
df['available_beds'] = np.random.randint(-25, 25, size=len(df))

In [ ]:
# Only keep rows with complete data
model_df = df[[
    'state_encoded', 'week_of_year', 'year',
    'admission_rate', 'emergency_surge',
    'staff_current_shortage', 'staff_anticipated_shortage',
    'available_beds', 'inpatient_beds_utilization'
]].dropna()

In [ ]:
staff_model_df = df.copy()
staff_model_df['target_staffing'] = staff_model_df.groupby('state')['staff_shortage_pct'].shift(-1)
staff_model_df = staff_model_df.dropna(subset=['target_staffing'])

X_staff = staff_model_df[[
    'state_encoded', 'week_of_year', 'year',
    'admission_rate', 'emergency_surge',
    'staff_current_shortage', 'staff_anticipated_shortage', 'available_beds'
]].astype(float)

y_staff = staff_model_df['target_staffing']from xgboost import XGBRegressor
import joblib

staff_model = XGBRegressor(n_estimators=100, random_state=42)
staff_model.fit(X_staff, y_staff)

In [ ]:
from xgboost import XGBRegressor
import joblib

staff_model = XGBRegressor(n_estimators=100, random_state=42)
staff_model.fit(X_staff, y_staff)

In [ ]:
joblib.dump(staff_model, "/Users/monica/Desktop/MSBA/2025 Spring/IDS560/project-group4/weekly tasks/week14-0419/ui/scenario_staff_model.pkl")

In [ ]:
X = model_df.drop(columns='inpatient_beds_utilization')
y = model_df['inpatient_beds_utilization']

In [ ]:
# Convert to float for XGBoost compatibility
X = X.astype(float)

In [ ]:
# Train and save
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X, y)

In [ ]:
# Save the trained model
joblib.dump(model, "/Users/monica/Desktop/MSBA/2025 Spring/IDS560/project-group4/weekly tasks/week14-0419/ui/scenario_xgb_model.pkl")
le = LabelEncoder().fit(df['state'])
joblib.dump(le, "/Users/monica/Desktop/MSBA/2025 Spring/IDS560/project-group4/weekly tasks/week14-0419/ui/scenario_state_encoder.pkl")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

feature_names = [
    'state_encoded', 'week_of_year', 'year', 'admission_rate',
    'emergency_surge', 'staff_current_shortage',
    'staff_anticipated_shortage', 'available_beds'
]

importances = model.feature_importances_
importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

plt.figure(figsize=(10, 5))
plt.barh(importance_df["Feature"], importance_df["Importance"])
plt.xlabel("Importance Score")
plt.title("XGBoost Feature Importances")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


# COVID-19 Prevalence & Impact

In [ ]:
from google.colab import files
import zipfile
import os
import pandas as pd

In [ ]:
uploaded1= files.upload()

In [ ]:
# Load Data
file_path = list(uploaded1.keys())[0]
df = pd.read_csv(file_path)

In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()
print("\nSummary Statistics:\n", df.describe())

In [ ]:
# Drop completely empty or redundant columns
drop_columns = [
    "geocoded_state",  # Completely empty column
    "inpatient_beds_utilization_numerator", "inpatient_beds_utilization_denominator", # Redundant
    "adult_icu_bed_utilization_numerator", "adult_icu_bed_utilization_denominator"
]
df = df.drop(columns=drop_columns, errors='ignore')


In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()

In [ ]:
# Drop the original detailed previous day admission columns
drop_admission_columns = [
    "previous_day_admission_adult_covid_confirmed_18-19", "previous_day_admission_adult_covid_confirmed_20-29",
    "previous_day_admission_adult_covid_confirmed_30-39", "previous_day_admission_adult_covid_confirmed_40-49",
    "previous_day_admission_adult_covid_confirmed_50-59", "previous_day_admission_adult_covid_confirmed_60-69",
    "previous_day_admission_adult_covid_confirmed_70-79", "previous_day_admission_adult_covid_confirmed_80+"
]
df = df.drop(columns=drop_admission_columns, errors='ignore')


In [ ]:
# Fill missing values
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

In [ ]:
# Convert 'date' column and extract temporal features
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['week_of_year'] = df['date'].dt.isocalendar().week
df = df.sort_values('date')



In [ ]:
numeric_df = df.select_dtypes(include=['number'])

# Compute correlation
corr_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, cmap="coolwarm", annot=False, fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
# Selecting required columns
columns = [
    "inpatient_beds_used_covid", "inpatient_beds_used",
    "deaths_covid", "date",
    "total_adult_patients_hospitalized_confirmed_and_suspected_covid",
    "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid",
    "total_patients_hospitalized_confirmed_influenza",
    "total_adult_patients_hospitalized_confirmed_covid"
]

df = df[columns].copy()

In [ ]:
# Handling missing values (replace zeros in denominators to avoid division by zero)
df.replace(0, np.nan, inplace=True)



In [ ]:
# Compute required features safely
df["COVID-19_inpatient_ratio"] = df["inpatient_beds_used_covid"] / df["inpatient_beds_used"]
df["Death_to_COVID_hospitalized_ratio"] = df["deaths_covid"] / (
    df["total_adult_patients_hospitalized_confirmed_and_suspected_covid"] +
    df["total_pediatric_patients_hospitalized_confirmed_and_suspected_covid"]
)
df["Influenza_vs_COVID_hospitalization_ratio"] = df["total_patients_hospitalized_confirmed_influenza"] / df["total_adult_patients_hospitalized_confirmed_covid"]

In [ ]:
# Remove inf values and NaNs caused by division errors
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
# Selecting only calculated metrics for visualization
features = [
    "COVID-19_inpatient_ratio",
    "Death_to_COVID_hospitalized_ratio",
    "Influenza_vs_COVID_hospitalization_ratio"
]


In [ ]:
# Summary statistics
print("Summary Statistics:\n", df[features].describe())

In [ ]:
#1. Histogram for Distribution
df[features].hist(figsize=(12, 6), bins=20, edgecolor="black")
plt.suptitle("Distribution of COVID-19 Prevalence & Impact Metrics", fontsize=14)
plt.show()

In [ ]:
import plotly.graph_objects as go

# Define features
features = [
    "COVID-19_inpatient_ratio",
    "Death_to_COVID_hospitalized_ratio",
    "Influenza_vs_COVID_hospitalization_ratio"
]

# Clip outliers
df_clipped = df.copy()
for col in features:
    low, high = df[col].quantile([0.01, 0.99])
    df_clipped[col] = df[col].clip(lower=low, upper=high)

# Prepare initial figure
fig = go.Figure()

# Add one trace per feature (we'll toggle visibility)
for i, col in enumerate(features):
    fig.add_trace(go.Histogram(
        x=df_clipped[col],
        name=col,
        visible=(i == 0),  # Show only the first one initially
        nbinsx=40
    ))

# Create dropdown buttons to toggle visibility
dropdown_buttons = [
    dict(label=col,
         method="update",
         args=[{"visible": [i == j for j in range(len(features))]},
               {"title": f"Distribution of {col} (Clipped)", "xaxis": {"title": col}}])
    for i, col in enumerate(features)
]

# Update layout
fig.update_layout(
    title=f"Distribution of {features[0]} (Clipped)",
    xaxis_title=features[0],
    updatemenus=[{
        "buttons": dropdown_buttons,
        "direction": "down",
        "x": 0.5,
        "xanchor": "center",
        "y": 1.2,
        "yanchor": "top"
    }],
    showlegend=False,
    template="plotly_white"
)

fig.show()


In [ ]:
#2. Correlation Heatmap
plt.figure(figsize=(8, 5))
sns.heatmap(df[features].corr(), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
#3 Line plot -Trends over time

import plotly.express as px
import pandas as pd

# Define your features
features = [
    "inpatient_beds_used_covid", "inpatient_beds_used", "deaths_covid",
    "total_adult_patients_hospitalized_confirmed_and_suspected_covid",
    "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid",
    "total_patients_hospitalized_confirmed_influenza",
    "total_adult_patients_hospitalized_confirmed_covid"
]

# Step 1: Ensure 'date' is datetime and sorted
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values('date')

# Step 2: Apply 7-day rolling average to smooth
df_smooth = df.copy()
for feature in features:
    df_smooth[feature] = df_smooth[feature].rolling(window=7, min_periods=1).mean()

# Step 3: Melt for long format
df_long = df_smooth.melt(id_vars="date", value_vars=features,
                         var_name="Metric", value_name="7-Day Avg")

# Step 4: Plot
fig = px.line(
    df_long,
    x="date",
    y="7-Day Avg",
    color="Metric",
    title="📈 7-Day Smoothed Trends of COVID-19 Metrics Over Time",
    labels={"date": "Date", "7-Day Avg": "Value", "Metric": "Metric"}
)

fig.update_layout(template="plotly_white")
fig.show()


In [ ]:
#4. Box Plots - Outliers Detection
import seaborn as sns
import matplotlib.pyplot as plt

# Clip outliers for visual clarity
df_clipped = df.copy()
for col in features:
    lower, upper = df[col].quantile([0.01, 0.99])
    df_clipped[col] = df[col].clip(lower=lower, upper=upper)

# Plot boxplot on clipped data
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_clipped[features])
plt.xticks(rotation=45)
plt.title("Clipped Boxplot of COVID-19 Prevalence & Impact Metrics (1st–99th Percentile)")
plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
#5. Violin Plots - Distribution & Density
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Violin Plots of COVID-19 Prevalence & Impact Metrics", fontsize=14)

for ax, feature in zip(axes, features):
    sns.violinplot(data=df[feature], ax=ax)
    ax.set_title(feature)

plt.tight_layout()
plt.show()

In [ ]:
#6. Bar Plot - Comparing Average Ratios**
plt.figure(figsize=(10, 6))
df[features].mean().sort_values().plot(kind="barh", color="lightcoral", edgecolor="black")
plt.xlabel("Average Value")
plt.ylabel("Feature Name")
plt.title("Average COVID-19 Prevalence & Impact Metrics")
plt.show()

In [ ]:
#7. Pairplot - Feature Relationships**
sns.pairplot(df[features])
plt.suptitle("Pairplot of COVID-19 Prevalence & Impact Metrics", fontsize=14)
plt.show()

**Admission and Discharge Trends**

In [ ]:
from google.colab import files
import zipfile
import os
import pandas as pd

In [ ]:
uploaded1= files.upload()

In [ ]:
# Load Data
file_path = list(uploaded1.keys())[0]
df = pd.read_csv(file_path)

In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()
print("\nSummary Statistics:\n", df.describe())

In [ ]:
# Drop completely empty or redundant columns
drop_columns = [
    "geocoded_state",  # Completely empty column
    "inpatient_beds_utilization_numerator", "inpatient_beds_utilization_denominator", # Redundant
    "adult_icu_bed_utilization_numerator", "adult_icu_bed_utilization_denominator"
]
df = df.drop(columns=drop_columns, errors='ignore')


In [ ]:
# Display basic information about the dataset
print("\nDataset Information:")
df.info()

In [ ]:
# Drop the original detailed previous day admission columns
drop_admission_columns = [
    "previous_day_admission_adult_covid_confirmed_18-19", "previous_day_admission_adult_covid_confirmed_20-29",
    "previous_day_admission_adult_covid_confirmed_30-39", "previous_day_admission_adult_covid_confirmed_40-49",
    "previous_day_admission_adult_covid_confirmed_50-59", "previous_day_admission_adult_covid_confirmed_60-69",
    "previous_day_admission_adult_covid_confirmed_70-79", "previous_day_admission_adult_covid_confirmed_80+"
]
df = df.drop(columns=drop_admission_columns, errors='ignore')


In [ ]:
# Fill missing values
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

In [ ]:
# Convert 'date' column and extract temporal features
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['week_of_year'] = df['date'].dt.isocalendar().week
df = df.sort_values('date')



In [ ]:
numeric_df = df.select_dtypes(include=['number'])

# Compute correlation
corr_matrix = numeric_df.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, cmap="coolwarm", annot=False, fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

**Admission & Discharge Trends**

In [ ]:
# Select relevant columns + keep date
columns = [
    "date",
    "previous_day_admission_adult_covid_confirmed",
    "inpatient_beds",
    "previous_day_admission_pediatric_covid_confirmed",
    "all_pediatric_inpatient_beds",
    "hospital_onset_covid",
    "total_adult_patients_hospitalized_confirmed_and_suspected_covid",
    "inpatient_beds_used",
    "month", "day_of_week", "week_of_year"
]
df = df[columns]



In [ ]:
# Handling missing values
print("Missing Values:\n", df.isnull().sum())

In [ ]:
# Compute new features based on given formulas
df["Daily_adult_admission_rate"] = df["previous_day_admission_adult_covid_confirmed"] / df["inpatient_beds"]
df["Daily_pediatric_admission_rate"] = df["previous_day_admission_pediatric_covid_confirmed"] / df["all_pediatric_inpatient_beds"]
df["Total_daily_admission_rate"] = (df["previous_day_admission_adult_covid_confirmed"] + df["previous_day_admission_pediatric_covid_confirmed"]) / df["inpatient_beds"]
df["Hospital_acquired_COVID_ratio"] = df["hospital_onset_covid"] / df["total_adult_patients_hospitalized_confirmed_and_suspected_covid"]
df["Discharge_to_admission_ratio"] = 1 - (df["hospital_onset_covid"] / df["inpatient_beds_used"])


In [ ]:
# Summary statistics
print("Summary Statistics:\n", df.describe())


In [ ]:
import matplotlib.dates as mdates

# Line plots for time series visualization
features = [
    "Daily_adult_admission_rate", "Daily_pediatric_admission_rate",
    "Total_daily_admission_rate", "Hospital_acquired_COVID_ratio",
    "Discharge_to_admission_ratio"
]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle("Time Series Trends of Admission & Discharge Metrics", fontsize=16)

for ax, feature in zip(axes.flatten(), features):
    sns.lineplot(data=df, x='date', y=feature, ax=ax)
    ax.set_title(feature)
    ax.set_xlabel("Date")
    ax.set_ylabel("Value")

    # Improve date formatting
    ax.xaxis.set_major_locator(mdates.YearLocator())  # Show tick every year
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format as Year-Month
    ax.tick_params(axis='x', rotation=45)  # Rotate x labels for readability

# Remove extra subplot if needed
if len(features) < len(axes.flatten()):
    fig.delaxes(axes.flatten()[-1])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:

# Compute 7-day moving averages for smoother trends
df["MA_7_Daily_adult_admission_rate"] = df["Daily_adult_admission_rate"].rolling(window=7).mean()
df["MA_7_Daily_pediatric_admission_rate"] = df["Daily_pediatric_admission_rate"].rolling(window=7).mean()
df["MA_7_Total_daily_admission_rate"] = df["Total_daily_admission_rate"].rolling(window=7).mean()
df["MA_7_Hospital_acquired_COVID_ratio"] = df["Hospital_acquired_COVID_ratio"].rolling(window=7).mean()
df["MA_7_Discharge_to_admission_ratio"] = df["Discharge_to_admission_ratio"].rolling(window=7).mean()


In [ ]:
import plotly.express as px

# First, melt the DataFrame to long format for Plotly
melted_df = df[[
    'date',
    'MA_7_Daily_adult_admission_rate',
    'MA_7_Daily_pediatric_admission_rate',
    'MA_7_Total_daily_admission_rate',
    'MA_7_Hospital_acquired_COVID_ratio',
    'MA_7_Discharge_to_admission_ratio'
]].melt(id_vars='date', var_name='Metric', value_name='Value')

# Plotly Interactive Line Plot
fig = px.line(
    melted_df,
    x="date",
    y="Value",
    color="Metric",
    labels={"Value": "7-Day Moving Average", "date": "Date"},
    title="📈 7-Day Moving Average Trends of Admission & Discharge Metrics"
)
fig.update_layout(template="plotly_white", legend_title_text='Metrics')
fig.show()


In [ ]:
# Correlation Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df[features].corr(), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

In [ ]:
#  Box Plots - Outliers Detection
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[features])
plt.xticks(rotation=45)
plt.title("Boxplot of Admission & Discharge Features (Detecting Outliers)")
plt.show()


In [ ]:
# --- Violin Plot ---
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle("Violin Plots of Admission & Discharge Trends", fontsize=14)

for ax, feature in zip(axes.flatten(), features):
    sns.violinplot(data=df, y=feature, ax=ax)
    ax.set_title(feature)

if len(features) < len(axes.flatten()):
    fig.delaxes(axes.flatten()[-1])

plt.tight_layout()
plt.show()


In [ ]:
#Pairplot - Relationship Between Features
fig = px.scatter_matrix(
    df,
    dimensions=features,
    color="month",  # Optional hue
    title="Scatter Matrix of Admission & Discharge Features"
)
fig.update_layout(width=2000, height=2000)
fig.show()

# 3. Modeling and Predictions

Splitting Data & Training a Baseline Model (Linear Regression Model)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = lr_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"📊 Model Performance:")
print(f"✅ Mean Absolute Error (MAE): {mae:.2f}")
print(f"✅ Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"✅ R-squared (R²): {r2:.2f}")


**Train a More Advanced Model (LSTM for Time-Series)**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape data for LSTM (samples, time steps, features)
X_train_lstm = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# Build LSTM Model
lstm_model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[1])),
    LSTM(50, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')

# Train the model
lstm_model.fit(X_train_lstm, y_train, epochs=20, batch_size=16, verbose=1)

# Make predictions
y_pred_lstm = lstm_model.predict(X_test_lstm)

# Evaluate
rmse_lstm = np.sqrt(mean_squared_error(y_test, y_pred_lstm))
print(f"✅ LSTM RMSE: {rmse_lstm:.2f}")


#To Improve Model Performance

**XGBoost Model**

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"📊 XGBoost Model Performance:")
print(f"✅ RMSE: {rmse_xgb:.2f}")
print(f"✅ R²: {r2_xgb:.2f}")


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'fit_intercept': [True, False]}
grid = GridSearchCV(LinearRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

print(f"Best Parameters: {grid.best_params_}")


**Fine-Tune XGBoost for Even Better Performance**

In [ ]:
xgb_tuned = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=3,
                         subsample=0.8, colsample_bytree=0.8,
                         reg_lambda=1, reg_alpha=0.5, random_state=42)
xgb_tuned.fit(X_train, y_train)

y_pred_xgb_tuned = xgb_tuned.predict(X_test)

rmse_xgb_tuned = np.sqrt(mean_squared_error(y_test, y_pred_xgb_tuned))
r2_xgb_tuned = r2_score(y_test, y_pred_xgb_tuned)

print(f"📊 Tuned XGBoost Performance:")
print(f"✅ RMSE: {rmse_xgb_tuned:.2f}")
print(f"✅ R²: {r2_xgb_tuned:.2f}")


In [ ]:
from sklearn.model_selection import cross_val_score

cv_rmse = np.sqrt(-cross_val_score(xgb_tuned, X_train, y_train, scoring="neg_mean_squared_error", cv=5))
print(f"✅ Cross-Validation RMSE: {cv_rmse.mean():.2f}")


**Interpret Model - Feature Importance**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


# Select ICU-related columns for prediction
features = ['staffed_adult_icu_bed_occupancy', 'total_staffed_adult_icu_beds',
            'inpatient_beds_used', 'inpatient_beds']  # Adjust based on relevant columns
target = 'staffed_adult_icu_bed_occupancy'  # Predicting ICU occupancy

# Normalize data
scaler = MinMaxScaler()
df_filtered[features] = scaler.fit_transform(df_filtered[features])

# Convert dataset into sequences
def create_sequences(data, target_col, seq_length=7):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Last 7 days as input
        y.append(target_col[i+seq_length])  # Fix: Directly index the target array
    return np.array(X), np.array(y)

# Apply the function
X, y = create_sequences(df_filtered[features].values, df_filtered[target].values)

# Reshape for CNN input (samples, time_steps, features)
X = X.reshape((X.shape[0], X.shape[1], len(features)))

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Define CNN Model
model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(7, len(features))),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(1)  # Output layer: Predict next day's ICU bed occupancy
])

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Evaluate Model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")


**Hybrid CNN-LSTM Model** for more improvement in CNN Model

In [ ]:
from tensorflow.keras.layers import LSTM, Reshape

model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(7, len(features))),
    MaxPooling1D(pool_size=2),
    LSTM(50, return_sequences=True),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
import xgboost
import tensorflow
print("XGBoost and TensorFlow are installed successfully!")



In [ ]:
# Train Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

In [ ]:
# Train XGBoost Model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

In [ ]:
# Build LSTM Model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])


In [ ]:
# Compile LSTM model
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')


In [ ]:
# Train LSTM model
lstm_model.fit(X_train_lstm, y_train, epochs=20, batch_size=32, validation_data=(X_test_lstm, y_test), verbose=1)


In [ ]:
# Predict with LSTM
lstm_predictions = lstm_model.predict(X_test_lstm)

In [ ]:
print("\n Model Training Completed! Predictions ready.")

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Function to evaluate models
def evaluate_model(model_name, y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n Performance of {model_name}:")
    print(f" RMSE: {rmse:.4f}")
    print(f" MAE: {mae:.4f}")
    print(f" R² Score: {r2:.4f}")
    return rmse, mae, r2

# Evaluate Random Forest
rf_metrics = evaluate_model("Random Forest", y_test, rf_predictions)

# Evaluate XGBoost
xgb_metrics = evaluate_model("XGBoost", y_test, xgb_predictions)

# Evaluate LSTM
lstm_metrics = evaluate_model("LSTM", y_test, lstm_predictions.flatten())  # Flatten to match shape


In [ ]:
# Ensure date column is available
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"])
    test_dates = df.iloc[y_test.index]["date"]
else:
    test_dates = range(len(y_test))  # Use indices if the date column is missing

def plot_predictions(y_true, y_pred, model_name, dates):
    plt.figure(figsize=(12, 6))
    plt.plot(dates, y_true.values, label="Actual", color='blue')
    plt.plot(dates, y_pred, label=f"Predicted ({model_name})", color='red', linestyle='dashed')

    plt.title(f"{model_name} Predictions vs Actual")
    plt.xlabel("Date" if isinstance(dates, pd.Series) else "Time Index")
    plt.ylabel("Inpatient Bed Utilization")
    plt.xticks(rotation=45)
    plt.legend()
    plt.show()

# Generate Plots with Fixed X-axis
plot_predictions(y_test, rf_predictions, "Random Forest", test_dates)
plot_predictions(y_test, xgb_predictions, "XGBoost", test_dates)
plot_predictions(y_test, lstm_predictions.flatten(), "LSTM", test_dates)



In [ ]:
# Forecast future inpatient bed utilization based on custom scenario
# Define a simple function for prediction using Random Forest or XGBoost
def predict_custom_utilization(model, model_name="Model"):
    print(f"\n Enter the scenario details for {model_name}:")

    # Define valid inputs
    seasons = ['Winter', 'Spring', 'Summer', 'Fall']
    admission_rates = [0, 25, 50]  # as percentages
    surge_levels = [0, 10, 20, 30]  # emergency surge levels
    staffing_options = ['Yes', 'No']
    bed_availability = [-25, 0, 25]  # % change in bed availability

    # User input with checks
    season = input(f"Season ({'/'.join(seasons)}): ")
    while season not in seasons:
        season = input(f"Invalid. Choose from {seasons}: ")

    admission_rate = int(input(f"Current Admission Rate % ({admission_rates}): "))
    while admission_rate not in admission_rates:
        admission_rate = int(input(f"Invalid. Choose from {admission_rates}: "))

    surge = int(input(f"Emergency Surge % ({surge_levels}): "))
    while surge not in surge_levels:
        surge = int(input(f"Invalid. Choose from {surge_levels}: "))

    staff_shortage = input(f"Staff Shortage (Yes/No): ")
    while staff_shortage not in staffing_options:
        staff_shortage = input(f"Invalid. Choose from {staffing_options}: ")

    bed_change = int(input(f"Available Beds Change % ({bed_availability}): "))
    while bed_change not in bed_availability:
        bed_change = int(input(f"Invalid. Choose from {bed_availability}: "))

    # Map categorical inputs to features (example encoding)
    season_map = {'Winter': 0, 'Spring': 1, 'Summer': 2, 'Fall': 3}
    staff_map = {'Yes': 1, 'No': 0}

    # Create a base input from mean values
    base_input = X.mean().copy()

    # Apply user scenario changes
    base_input['month'] = season_map[season] * 3 + 1
    base_input['staffing_shortage_indicator'] = staff_map[staff_shortage]
    base_input['total_previous_day_admissions'] *= (1 + admission_rate / 100)
    base_input['inpatient_beds'] *= (1 + bed_change / 100)
    base_input['hospital_onset_covid'] *= (1 + surge / 100)

    # Prepare and reshape input
    input_df = pd.DataFrame([base_input])
    prediction = model.predict(input_df)[0]
    print(f"\n Predicted Inpatient Bed Utilization using {model_name}: {prediction:.4f}")

# Use both models
predict_custom_utilization(rf_model, model_name="Random Forest")
predict_custom_utilization(xgb_model, model_name="XGBoost")

In [ ]:
import itertools

# Batch Simulation Function
def batch_simulation(models):
    print("\n Running Batch Simulations for All Scenario Combinations...")

    seasons = ['Winter', 'Spring', 'Summer', 'Fall']
    admission_rates = [0, 25, 50]
    surge_levels = [0, 10, 20, 30]
    staffing_options = ['Yes', 'No']
    bed_availability = [-25, 0, 25]

    season_map = {'Winter': 0, 'Spring': 1, 'Summer': 2, 'Fall': 3}
    staff_map = {'Yes': 1, 'No': 0}

    results = []
    combinations = list(itertools.product(seasons, admission_rates, surge_levels, staffing_options, bed_availability))

    for (season, admission_rate, surge, staff_shortage, bed_change) in combinations:
        scenario = {
            'month': season_map[season] * 3 + 1,
            'admission_rate': admission_rate,
            'surge': surge,
            'staff_shortage': staff_map[staff_shortage],
            'bed_change': bed_change
        }

        base_input = X.mean().copy()
        base_input['month'] = scenario['month']
        base_input['staffing_shortage_indicator'] = scenario['staff_shortage']
        base_input['total_previous_day_admissions'] *= (1 + scenario['admission_rate'] / 100)
        base_input['inpatient_beds'] *= (1 + scenario['bed_change'] / 100)
        base_input['hospital_onset_covid'] *= (1 + scenario['surge'] / 100)

        input_df = pd.DataFrame([base_input])

        row = {
            'Season': season,
            'Admission Rate %': admission_rate,
            'Emergency Surge %': surge,
            'Staff Shortage': staff_shortage,
            'Bed Availability %': bed_change
        }

        for model_name, model in models.items():
            row[f'{model_name} Prediction'] = model.predict(input_df)[0]

        results.append(row)

    result_df = pd.DataFrame(results)
    print("\n Batch Simulations Completed. Showing First 10 Results:")
    print(result_df.head(10))

    # Visualization
    import seaborn as sns
    import matplotlib.pyplot as plt

    for model_name in models:
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=result_df, x='Season', y=f'{model_name} Prediction', hue='Staff Shortage')
        plt.title(f"{model_name} Predictions by Season and Staff Shortage")
        plt.ylabel("Predicted Bed Utilization")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    return result_df

# Run batch simulation
batch_simulation({"Random Forest": rf_model, "XGBoost": xgb_model})



In [ ]:
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
models = {
    "Random Forest": rf_model,
    "XGBoost": xgb_model
}

result_df = batch_simulation(models)
# Ensure 'Season' and 'Staff Shortage' are strings for plotting
result_df['Season'] = result_df['Season'].astype(str)
result_df['Staff Shortage'] = result_df['Staff Shortage'].astype(str)

for model_name in models:

        # Boxplot by Season and Staff Shortage
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=result_df, x='Season', y=f'{model_name} Prediction', hue='Staff Shortage')
        plt.title(f"{model_name} Predictions by Season and Staff Shortage")
        plt.ylabel("Predicted Bed Utilization")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

        # Lineplot for Admission Rate
        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=result_df[result_df['Staff Shortage'] == 'No'],
            x='Admission Rate %',
            y=f'{model_name} Prediction',
            hue='Season',
            marker='o'
        )
        plt.title(f"{model_name} Predictions vs Admission Rate (No Staff Shortage)")
        plt.ylabel("Predicted Bed Utilization")
        plt.xlabel("Admission Rate %")
        plt.tight_layout()
        plt.show()

        # Lineplot for Bed Availability
        plt.figure(figsize=(10, 6))
        sns.lineplot(
            data=result_df[result_df['Staff Shortage'] == 'No'],
            x='Bed Availability %',
            y=f'{model_name} Prediction',
            hue='Season',
            marker='o'
        )
        plt.title(f"{model_name} Predictions vs Bed Availability (No Staff Shortage)")
        plt.ylabel("Predicted Bed Utilization")
        plt.xlabel("Bed Availability %")
        plt.tight_layout()
        plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate RMSE and R² for each model
metrics = {
    "Random Forest": {
        "RMSE": np.sqrt(mean_squared_error(y_test, rf_predictions)),
        "R²": r2_score(y_test, rf_predictions)
    },
    "XGBoost": {
        "RMSE": np.sqrt(mean_squared_error(y_test, xgb_predictions)),
        "R²": r2_score(y_test, xgb_predictions)
    },
    "LSTM": {
        "RMSE": np.sqrt(mean_squared_error(y_test, lstm_predictions.flatten())),
        "R²": r2_score(y_test, lstm_predictions.flatten())
    }
}

# Convert to DataFrame
metric_df = pd.DataFrame(metrics).T.reset_index().rename(columns={"index": "Model"})

# Melt the dataframe for seaborn plotting
metric_melted = metric_df.melt(id_vars="Model", var_name="Metric", value_name="Score")

# Plot grouped bar chart
plt.figure(figsize=(8, 6))
sns.barplot(data=metric_melted, x="Model", y="Score", hue="Metric")
plt.title("Model Performance: RMSE vs R²")
plt.ylabel("Score")
plt.xlabel("Model")
plt.ylim(0, 1)  # Normalize scale for comparison
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# Calculate RMSE and R² for each model
rmse_scores = {
    "Random Forest": np.sqrt(mean_squared_error(y_test, rf_predictions)),
    "XGBoost": np.sqrt(mean_squared_error(y_test, xgb_predictions)),
    "LSTM": np.sqrt(mean_squared_error(y_test, lstm_predictions.flatten())),
}

r2_scores = {
    "Random Forest": r2_score(y_test, rf_predictions),
    "XGBoost": r2_score(y_test, xgb_predictions),
    "LSTM": r2_score(y_test, lstm_predictions.flatten()),
}

# Prepare data
models = list(rmse_scores.keys())
x = np.arange(len(models))  # label locations
width = 0.35  # width of bars

# Plot
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot RMSE
rmse_vals = list(rmse_scores.values())
r2_vals = list(r2_scores.values())

bar1 = ax1.bar(x - width/2, rmse_vals, width, label='RMSE', color='skyblue')
ax1.set_ylabel('RMSE', color='skyblue')
ax1.set_xlabel('Model')
ax1.set_title('📊 RMSE vs R² for Each Model')
ax1.set_xticks(x)
ax1.set_xticklabels(models)

# Create second y-axis for R²
ax2 = ax1.twinx()
bar2 = ax2.bar(x + width/2, r2_vals, width, label='R²', color='lightgreen')
ax2.set_ylabel('R² Score', color='lightgreen')
ax2.set_ylim(0, 1.1)

# Annotate bars
for i in range(len(models)):
    ax1.text(x[i] - width/2, rmse_vals[i] + 0.005, f"{rmse_vals[i]:.3f}", ha='center', color='blue')
    ax2.text(x[i] + width/2, r2_vals[i] + 0.02, f"{r2_vals[i]:.3f}", ha='center', color='green')

# Legends
fig.legend(loc="upper center", ncol=2)
plt.tight_layout()
plt.show()

# 6. Staffing Bed Prediction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Loading the dataset
file_path = "C:/Users/nehap/Downloads/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries__RAW__20250315_Updated.csv"

# Load dataset
df = pd.read_csv(file_path)

In [ ]:
df['Staff Availability %'] = (
    df['critical_staffing_shortage_today_no'] /
    (df['critical_staffing_shortage_today_yes'] + df['critical_staffing_shortage_today_no']).replace(0, np.nan)
) * 100


In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
# Drop irrelevant or non-numeric columns, include engineered Staff Availability %
features = df.drop(columns=['Staff Availability %', 'date', 'state'])
target = df['Staff Availability %']

# Drop rows where Staff Availability % is NaN
df = df.dropna(subset=['Staff Availability %'])

# Define features and target again after cleaning
features = df.drop(columns=['Staff Availability %', 'date', 'state'])  # drop non-numeric/categorical
target = df['Staff Availability %']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)




In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)

print("RF RMSE:", mean_squared_error(y_test, pred_rf, squared=False))


In [ ]:
import pandas as pd

# Ensure date column is datetime
df['date'] = pd.to_datetime(df['date'])

# Aggregate weekly staff availability by state
df_weekly = df.groupby(['state', pd.Grouper(key='date', freq='W-MON')])['Staff Availability %'].mean().reset_index()

# Sort by state and date
df_weekly = df_weekly.sort_values(by=['state', 'date'])

# Add lag features (e.g., 1 week, 2 weeks ago)
for lag in [1, 2, 3]:
    df_weekly[f'lag_{lag}'] = df_weekly.groupby('state')['Staff Availability %'].shift(lag)

# Drop rows with missing lags
df_weekly = df_weekly.dropna()


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Features and target
feature_cols = ['lag_1', 'lag_2', 'lag_3']
X = df_weekly[feature_cols]
y = df_weekly['Staff Availability %']

# Train the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, y)


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px


In [ ]:
# Load your dataset (change path accordingly)
df = pd.read_csv("C:/Users/nehap/Downloads/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries__RAW__20250315_Updated.csv")

# Ensure datetime
df['date'] = pd.to_datetime(df['date'])

# Keep necessary columns only
df = df[['date', 'state', 'critical_staffing_shortage_today_yes', 'critical_staffing_shortage_today_no']]

# Calculate Staff Availability %
df['Staff Availability %'] = (
    df['critical_staffing_shortage_today_no'] /
    (df['critical_staffing_shortage_today_yes'] + df['critical_staffing_shortage_today_no'])
) * 100

# Drop any rows with missing data
df = df.dropna()


In [ ]:
# Weekly state-level aggregation
df_weekly = df.groupby(['state', pd.Grouper(key='date', freq='W-MON')])['Staff Availability %'].mean().reset_index()

# Sort and add lag features
df_weekly = df_weekly.sort_values(by=['state', 'date'])
for lag in [1, 2, 3]:
    df_weekly[f'lag_{lag}'] = df_weekly.groupby('state')['Staff Availability %'].shift(lag)

df_weekly = df_weekly.dropna()


In [ ]:
X = df_weekly[['lag_1', 'lag_2', 'lag_3']]
y = df_weekly['Staff Availability %']

model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X, y)


In [ ]:
fig = px.choropleth(
    future_df,
    locations='state',
    locationmode='USA-states',
    color='Predicted Staff Availability %',
    hover_name='state',
    animation_frame='date_str',
    scope='usa',
    color_continuous_scale='Viridis',
    title='8-Week Predicted Staff Availability % (Post-April 2024)'
)

fig.update_layout(
    geo=dict(bgcolor='rgba(0,0,0,0)'),
    margin={"r":0,"t":50,"l":0,"b":0},
    coloraxis_colorbar=dict(title="Availability %")
)

fig.show()


# 5. State-Wise Bed Utilization and XGBoost Forecasting

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Sample placeholder for your dataset loading
df = pd.read_csv('/Users/ng/cleaned_hospital_data.csv')  # Replace with actual path

# Assuming 'state' column exists and 'bed_utilization_ratio' represents bed usage
state_util = df.groupby('state')['inpatient_beds_utilization'].mean().reset_index()

# Plotting bar chart of state-wise average bed utilization
plt.figure(figsize=(14, 8))
state_util_sorted = state_util.sort_values(by='inpatient_beds_utilization', ascending=False)
sns.barplot(data=state_util_sorted, y='state', x='inpatient_beds_utilization', palette='coolwarm')
plt.title("Average State-Wise Bed Utilization Ratio")
plt.xlabel("Bed Utilization Ratio")
plt.ylabel("State")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Load your cleaned dataset
df = pd.read_csv('/Users/ng/cleaned_hospital_data.csv')  # Update with your path

# Group by state
state_util = df.groupby('state')['inpatient_beds_utilization'].mean().reset_index()
state_util.columns = ['state', 'avg_utilization']

# Create enhanced choropleth map
fig = px.choropleth(
    state_util,
    locations='state',
    locationmode='USA-states',
    color='avg_utilization',
    color_continuous_scale='RdBu_r',  # Reversed for blue = high
    scope='usa',
    hover_data={'state': True, 'avg_utilization': ':.3f'},
    labels={'avg_utilization': 'Avg. Bed Utilization'},
    title='State-wise Average Bed Utilization Ratio (Interactive)'
)

# Enhance layout and clarity
fig.update_layout(
    title_font_size=22,
    geo=dict(
        showlakes=True,
        lakecolor='rgb(255, 255, 255)',
        bgcolor='rgba(0,0,0,0)'
    ),
    coloraxis_colorbar=dict(
        title='Utilization Ratio',
        tickformat='.2f',
        lenmode='pixels', len=250,
        thickness=15
    ),
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Show figure
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import ipywidgets as widgets
from IPython.display import display

# Load dataset
df = pd.read_csv('/Users/ng/cleaned_hospital_data.csv')
df['year'] = 2022  # Add dummy year
le = LabelEncoder()
df['state_encoded'] = le.fit_transform(df['state'])

features = ['state_encoded', 'week_of_year', 'year']

def predict_and_plot(weeks_ahead):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(['state', 'week_of_year'])
    df_copy['target_util'] = df_copy.groupby('state')['inpatient_beds_utilization'].shift(-weeks_ahead)
    model_df = df_copy.dropna(subset=['target_util'])

    X = model_df[features]
    y = model_df['target_util']

    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X, y)

    latest_week_data = df.groupby('state').tail(1).copy()
    latest_week_data['week_of_year'] += weeks_ahead
    latest_week_data['state_encoded'] = le.transform(latest_week_data['state'])
    pred_input = latest_week_data[features]
    latest_week_data['predicted_util'] = model.predict(pred_input)

    fig = px.choropleth(
        latest_week_data,
        locations='state',
        locationmode='USA-states',
        color='predicted_util',
        scope='usa',
        color_continuous_scale='RdBu_r',
        hover_data={'state': True, 'predicted_util': ':.3f'},
        labels={'predicted_util': f'Prediction (t+{weeks_ahead}w)'},
        title=f"🛏️ Predicted State-wise Bed Utilization ({weeks_ahead} Weeks Ahead)"
    )

    fig.update_geos(
    fitbounds="locations",
    visible=False,
    projection_scale=5.8,  # tighter zoom
    center={"lat": 38, "lon": -96},  # center on continental US
    )

    fig.update_layout(
        title_font_size=24,
        height=650,  # taller for better map size
        geo=dict(
            showlakes=True,
            lakecolor='rgb(255, 255, 255)',
            showland=True,
            landcolor='rgb(240,240,240)',
            subunitcolor='black',
            showframe=False,
            showcountries=False,
            ),
    margin=dict(r=20, t=60, l=20, b=20),
    coloraxis_colorbar=dict(
        title='Utilization Ratio',
        tickformat='.2f',
        len=300,
        thickness=18,
        xpad=10
        )
      )

fig.show()

# Interactive widget
slider = widgets.IntSlider(value=2, min=1, max=8, step=1, description='Weeks Ahead:')
widgets.interact(predict_and_plot, weeks_ahead=slider)
